## Lectura de archivos de alineamientos múltiples de secuencias.

El módulo Bio.AlignIO contiene funciones que permiten leer archivos de MSA.

La función *read* lee el contenido de un archivo.
Se lo puede convertir en una lista para acceder a cada secuencia.
Es necesario especificar el formato de los archivos de entrada.

In [29]:
from Bio import AlignIO

seqs = AlignIO.read("data/some_sequences.fasta", format="fasta")
seqs = list(seqs)
print(seqs)

[SeqRecord(seq=Seq('ACATCGACTGACTG', SingleLetterAlphabet()), id='s1', name='s1', description='s1', dbxrefs=[]), SeqRecord(seq=Seq('CATACTATCTCTAT', SingleLetterAlphabet()), id='s2', name='s2', description='s2', dbxrefs=[]), SeqRecord(seq=Seq('ACATACTATCTCTA', SingleLetterAlphabet()), id='s3', name='s3', description='s3', dbxrefs=[])]


Cada secuencia individual se representa como un objeto de tipo *SeqRecord*.

* Un *SeqRecord* tiene dos atributos importantes: *id* y *seq*. 
 * El *id* es un string que contiene el nombre o identificador de la secuencia, 
 * y *seq* es un objeto de tipo *Seq* que contiene los datos de la secuencia.
* Un objeto *Seq* tiene la secuencia propiamente dicha y un alfabeto.


In [28]:
s1 = seqs[0]
print("s1 es la primer secuencia")
print("s1 es de tipo: " + str(type(s1)))
print("El id de s1 es: " + s1.id)
print("La secuencia de s1 es de tipo: " + str(type(s1.seq)))
print("La secuencia de s1 es: " + s1.seq)
print("La secuencia de s1 tiene un el alfabeto: " + str(s1.seq.alphabet))

s1 es la primer secuencia
s1 es de tipo: <class 'Bio.SeqRecord.SeqRecord'>
El id de s1 es: s1
La secuencia de s1 es de tipo: <class 'Bio.Seq.Seq'>
La secuencia de s1 es: ACATCGACTGACTG
La secuencia de s1 tiene un el alfabeto: SingleLetterAlphabet()


### Crear un objeto *Seq*

In [65]:
from Bio.Seq import Seq
from Bio.Seq import translate
from Bio.Alphabet import DNAAlphabet

new_seq = Seq("ACTACTGACTAC", DNAAlphabet())
print(new_seq)

ACTACTGACTAC


### Exportar secuencias a un archivo

In [70]:
from Bio import AlignIO
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

s1 = SeqRecord(id="Secuencia1", seq=Seq("ACTATCTAGCTACTG"))
s2 = SeqRecord(id="Secuencia2", seq=Seq("CTATCTAGCTACTGA"))
s3 = SeqRecord(id="Secuencia3", seq=Seq("TATCTAGCTACTGAA"))
s4 = SeqRecord(id="Secuencia4", seq=Seq("ATCTAGCTACTGAAA"))

secuencias = [s1, s2, s3, s4]

SeqIO.write(secuencias, 'secuencias_de_ejemplo.fasta', format="fasta")
SeqIO.write(secuencias, 'secuencias_de_ejemplo.clustal', format="clustal")

# acepta muchos otros formato:
# phylip, nexus, stockholm, etc

4

### Convertir el formato de un archivo

La función *convert*

In [71]:
SeqIO.convert('secuencias_de_ejemplo.fasta', 'fasta', 'secuencias_de_ejemplo.stockholm', 'stockholm')

4

### Operaciones sobre secuencias

In [63]:
from Bio.Seq import Seq
from Bio.Seq import translate
from Bio.Alphabet import DNAAlphabet

new_seq = Seq("ATGTTAGCTTGA", DNAAlphabet())

translated = new_seq.translate()
print("translated : " + translated)

transcribed = new_seq.transcribe()
print("transcribed: "+transcribed)

complement = new_seq.complement()
print("complement : "+complement)

rev_complement = new_seq.reverse_complement()
print("complement : "+rev_complement)

new_seq = Seq("---ATGTTAGCTTGA---", DNAAlphabet())
degapped = new_seq.ungap("-")
print("ungapped : "+degapped)

translated : MLA*
transcribed: AUGUUAGCUUGA
complement : TACAATCGAACT
complement : TCAAGCTAACAT
ungapped : ATGTTAGCTTGA


## Leer archivos PDB

Un PDB tiene una jerarquía de elementos que definen cada componente de la estuctura 3D de una mocromolécula.

* Un PDB -> Contiene una Estructura
* Estructura -> Contiene uno o más modelos. 
* Modelo -> Contiene una o más cadenas.
* Cadena -> Contiene uno o más residuos.
* Residuo -> Contiene uno o más átomos.
* Átomo -> Elemeton base de la jerarquía.

In [89]:
from Bio.PDB import PDBParser

parser = PDBParser()
structure = parser.get_structure('6H9U', 'data/6h9u.pdb')

for model in structure:
    for chain in model:
        for residue in chain:
            for atom in residue:
                print(atom)
                



<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom OG1>
<Atom CG2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG1>
<Atom CG2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG1>
<Atom CG2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG1>
<Atom CG2>
<Atom CD1>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom OD1>
<Atom OD2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD1>
<Atom CD2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom OG1>
<Atom CG2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom OG1>
<Atom CG2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD1>
<Atom CD2>
<Atom CE1>
<Atom CE2>
<Atom CZ>
<Atom OH>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom OG>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom SG>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG1>
<Atom CG2>
<A

<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom SG>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom SG>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD>
<Atom OE1>
<Atom OE2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD>
<Atom CE>
<Atom NZ>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom OG>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom OD1>
<Atom OD2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD1>
<Atom CD2>
<Atom CE1>
<Atom CE2>
<Atom CZ>
<Atom OH>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG1>
<Atom CG2>
<Atom CD1>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD>
<Atom NE>
<Atom CZ>
<Atom NH1>
<Atom NH2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD>
<Atom CE>
<Atom NZ>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG

### Calcular distancia entre dos átomos

In [85]:
atom1 = structure[0]['A'][100]['CA']
atom2 = structure[0]['A'][99]['CA']
print(atom1, atom2, atom1-atom2)

<Atom CA> <Atom CA> 3.7955189
